In [ ]:
import numpy as np
import pandas as pd
import pmdarima as pm

from pmdarima.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt
import os

#Read in the data

In [ ]:
sales_data = pd.read_csv("Tractor-Sales.csv")
sales_data.head(5)

#Since the complete date is not mentioned, assume the first of every month hence freq = 'MS' (month start)

In [ ]:
dates = pd.date_range(start='2003-01-01', freq='MS', periods=len(sales_data))

In [ ]:
import calendar
sales_data['Month'] = dates.month
sales_data['Month'] = sales_data['Month'].apply(lambda x: calendar.month_abbr[x])
sales_data['Year'] = dates.year

In [ ]:
sales_data.drop(['Month-Year'], axis=1, inplace=True)
sales_data.rename(columns={'Number of Tractor Sold':'Tractor-Sales'}, inplace=True)
sales_data = sales_data[['Month', 'Year', 'Tractor-Sales']]

#set the dates as the index of the dataframe so it can be treated as a time-series dataframe

In [ ]:
sales_data.set_index(dates, inplace=True)
sales_data.head(5)

#plot the time-series

In [ ]:
sales_ts = sales_data['Tractor-Sales']
plt.figure(figsize=(10, 5))
plt.plot(sales_ts)
plt.xlabel('Years')
plt.ylabel('Tractor Sales')

#decomposition of the time series
#It is multiplicative because the amplitude of the seasonal variances increases with time

In [ ]:
decomposition = seasonal_decompose(sales_ts, model='multiplicative', period=12)
decomposition.plot()

In [ ]:
# split into train and test, the last 12 months to test
train = sales_ts.iloc[:132]
test = sales_ts.iloc[132:]

In [ ]:
# auto ARIMA
model_auto_arima = pm.auto_arima(train, m=12)
model_auto_arima.summary()

In [ ]:
# Predict the last 12 months of the original data
forecast_auto_arima = model_auto_arima.predict(12)

In [ ]:
# Forecast versus Actual for the test data
plt.figure(figsize = (20,7))
plt.title('Tractor Sales')
x = np.arange(sales_data.shape[0])
plt.plot(sales_data.index[:132], train, c='blue')
plt.plot(sales_data.index[132:], forecast_auto_arima, c='green', label = 'Auto ARIMA Forecast')
plt.plot(sales_data.index[132:], test, c='red', label = 'Actual')
plt.legend()
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Mean Absolute Error (MAE)
MAE = (abs(forecast_auto_arima - test)).mean()
print('The Mean Absolute Error of the auto arima forecasts is', round(MAE, 2))

In [ ]:
# Mean Absolute Percentage Error (MAPE)
MAPE = (abs(forecast_auto_arima - test)/abs(test)).mean()
print('The Mean Absolute Percentage Error of the auto arima forecasts is', round(MAPE, 4))

### Now let's do it on our own, without auto arima

##### Using ACF and PACF to find the model orders.

###### First, we need to make the time series stationary.

Check stationarity using Dickey-Fuller test
Dickey-Fuller test assumes null hypothesis non-stationary time series

In [ ]:
from statsmodels.tsa.stattools import adfuller
print('Results of Dickey-Fuller Test:')
dftest = adfuller(sales_ts, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

differencing to stationarise the time series

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(sales_ts.diff(periods=1))
plt.xlabel('Years')
plt.ylabel('Tractor Sales')

The data becomes stationary by mean (flat) but not stationary by variance (the amplitude of the variance increases).

To make a time series stationary on variance it is best to first perform a logarithmic transformation of the original data.

Plot of the logarithm of the original data (log10) 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.log10(sales_ts))
plt.xlabel('Years')
plt.ylabel('Log (Tractor Sales)')

The series of the logarithms of the data is stationary in variance but not in mean

Difference the log-transformed series

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.log10(sales_ts).diff(periods=1))
plt.xlabel('Years')
plt.ylabel('Differenced Log (Tractor Sales)')

This looks stationary in both mean and variance

In [ ]:
sales_ts_log = np.log10(sales_ts)
sales_ts_log.dropna(inplace=True)

sales_ts_log_diff = sales_ts_log.diff(periods=1) 
sales_ts_log_diff.dropna(inplace=True)

Dickey-Fuller test 

In [ ]:
print('Results of Dickey-Fuller Test:')
dftest = adfuller(sales_ts_log_diff, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

autocorrelation function (ACF) and partial autocorrelation function (PACF)

In [ ]:
acf = plot_acf(sales_ts_log_diff)
plt.show()

In [ ]:
pacf = plot_pacf(sales_ts_log_diff)
plt.show()

# Fit ARIMA model

##### ARIMA(p,d,q)(P,D,Q)12
##### d=1 (differenced once)
##### p=0, q=0 (nothing significant after lag 0)
##### D=0 (not differenced across seasons)
##### P=1, Q=0 (decay in the seasonal lags of PACF, single drop in ACF)

In [ ]:
train_log = np.log10(train)
train_log.dropna(inplace=True)

test_log = np.log10(test)
test_log.dropna(inplace=True)

In [ ]:
model_ARIMA = pm.ARIMA(order=(0,1,0),seasonal_order=(1,0,0,12))
model_ARIMA_fit=model_ARIMA.fit(train_log)

# Generate predictions (forecasts) 12 periods ahead
forecast_ARIMA = model_ARIMA_fit.predict(12)

##### The model is on the logarithm of sales at base 10. Let us convert the values. We need to take the exponent of the logarithm - this is the opposite function, the antilogarithm.

In [ ]:
forecast_ARIMA = 10**forecast_ARIMA

In [ ]:
# Forecast versus Actual for the test data 
plt.figure(figsize = (20,7))
plt.title('Tractor Sales')
x = np.arange(sales_data.shape[0])
plt.plot(sales_data.index[:132], train, c='blue')
plt.plot(sales_data.index[132:], forecast_ARIMA, c='green', label = 'ARIMA Forecast')
plt.plot(sales_data.index[132:], test, c='red', label = 'Actual')
plt.legend()
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Mean Absolute Error (MAE)
MAE = (abs(forecast_ARIMA - test)).mean()
print('The Mean Absolute Error of our forecasts is',round(MAE, 2))

In [ ]:
# Mean Absolute Percentage Error (MAPE)
MAPE = (abs(forecast_ARIMA - test)/abs(test)).mean()
print('The Mean Absolute Percentage Error of our forecasts is',round(MAPE, 4))